# サンプルサイズが異なる群の比較で何がおきるだろうか？

## 言葉
- 第 1 種の過誤 (Type I error)
    - 有意水準が `α` のときに、実際には有意差がないのに有意差があると判断してしまうこと。
- 第 2 種の過誤 (Type II error)
    - 実際には有意差があるのに有意差がないと判断してしまうこと。この確率を `β` で表す。
- 検定力
    - 正しく有意差を検出できる確率。`1 - β` で表す。

# 調査内容
[Controlled experiments on the web: survey and practical guide](Controlled experiments on the web: survey and practical guide) のサンプルを使って、ある一定の設定での比較のシミュレートを行う。

>　If, however, you were only looking for a 5% change in conversion rate (not revenue), a lower variability OEC based on point 3.b can be used. Purchase, a conversion event, is modeled as a Bernoulli trial with p = 0.05 being the probability of a purchase. The standard deviation of a Bernoulli is √ p(1 − p) and thus you will need less than 122,000 users to achieve the desired power based on 16 ∗ (0.05 * (1 − 0.05))/(0.05 *
0.05) ^2.

- 比較設定
    - それぞれのサンプルサイズ: `122,000` (合計 `244,000`)
    - 平均: `0.05`
    - 検出したい変化: `5%` の変化
    - 比較する指標の分布: 二項分布
    - 有意水準 5% (`α = 0.05`)
    - 検出力 0.8 (`β = 0.2`)

上記の設定での 2 群の比較を χ 二乗検定で行なった時、サンプルサイズを変化させた時に何が起きるのかを確認する。力技で。

In [1]:
def is_significant_difference(size_a: int, size_b: int, ratio_a: float = 0.05, ratio_b: float = 0.05, alpha:float = 0.05):
    a = np.random.binomial(size_a, ratio_a, 1)[0]
    b = np.random.binomial(size_b, ratio_b, 1)[0]
    return stats.chi2_contingency([[size_a - a, a],[size_b - b, b]])[1] < alpha

sample_size = 122000
avg = 0.05
lift = 0.05
trial = 100000

## 1. 実際に有意差が存在する場合
- 群 A の平均 `0.05`
- 群 B の平均 `0.05 + 5% UP`

上記の比較を 10 万回行なった時、正しく有意差が検出された割合を調べる

## そのままの設定で比較

In [2]:
result = Parallel(n_jobs=-1, verbose=0)([delayed(is_significant_difference)(sample_size, sample_size, ratio_a = avg, ratio_b = avg + avg*lift) for _ in range(trial)])
sum([1 if x else 0 for x in result])/trial

0.79666

検出力に設定した `0.8` がそのまま現れている（20% の確率で Type II Error が起きている）

## サンプルサイズをそれぞれ倍に

In [3]:
result = Parallel(n_jobs=-1, verbose=0)([delayed(is_significant_difference)(sample_size *2, sample_size *2, ratio_a = avg, ratio_b = avg + avg*lift) for _ in range(trial)])
sum([1 if x else 0 for x in result])/trial

0.97721

サンプルサイズが上がったことで、観測値の確度が上がり、Type II Error が殆ど発生していない。

## 群 A のサンプルサイズを半分にし、減らした分を群 B に移した場合（不均衡な比較）

In [4]:
result = Parallel(n_jobs=-1, verbose=0)([delayed(is_significant_difference)(sample_size *0.5, sample_size *1.5, ratio_a = avg, ratio_b = avg + avg*lift) for _ in range(trial)])
sum([1 if x else 0 for x in result])/trial

0.67585

サンプルサイズの合計が同じでも群間でサンプルサイズに差がある場合、当初の設定である検出力を満たせなくなる。

## 群 B のサンプルサイズを半分にし、減らした分を群 A に移した場合（不均衡な比較）

In [5]:
result = Parallel(n_jobs=-1, verbose=0)([delayed(is_significant_difference)(sample_size *1.5, sample_size *0.5, ratio_a = avg, ratio_b = avg + avg*lift) for _ in range(trial)])
sum([1 if x else 0 for x in result])/trial

0.67395

どちらの群のサイズが変化しても傾向は同じ（検出力は落ちる）

## 2. 実際には有意差が存在しない場合
- 群 A の平均 `0.05`
- 群 B の平均 `0.05`

上記の比較を 10 万回行なった時、正しく有意差が検出された割合を調べる

## そのままの設定で比較

In [6]:
result = Parallel(n_jobs=-1, verbose=0)([delayed(is_significant_difference)(sample_size, sample_size, ratio_a = avg, ratio_b = avg) for _ in range(trial)])
sum([1 if x else 0 for x in result])/trial

0.04895

有意水準に設定した 0.05 が現れる。

## サンプルサイズをそれぞれ倍に

In [7]:
result = Parallel(n_jobs=-1, verbose=0)([delayed(is_significant_difference)(sample_size *2, sample_size *2, ratio_a = avg, ratio_b = avg) for _ in range(trial)])
sum([1 if x else 0 for x in result])/trial

0.04992

サンプルサイズが上がったことで、観測値の確度が上がり、Type II Error が殆ど発生していない。

## 群 A のサンプルサイズを半分にし、減らした分を群 B に移した場合（不均衡な比較）

In [8]:
result = Parallel(n_jobs=-1, verbose=0)([delayed(is_significant_difference)(sample_size *0.5, sample_size *1.5, ratio_a = avg, ratio_b = avg) for _ in range(trial)])
sum([1 if x else 0 for x in result])/trial

0.04858

サンプルサイズの合計が同じでも群間でサンプルサイズに差がある場合、当初の設定である検出力を満たせなくなる。

## 群 B のサンプルサイズを半分にし、減らした分を群 A に移した場合（不均衡な比較）

In [9]:
result = Parallel(n_jobs=-1, verbose=0)([delayed(is_significant_difference)(sample_size *1.5, sample_size *0.5, ratio_a = avg, ratio_b = avg) for _ in range(trial)])
sum([1 if x else 0 for x in result])/trial

0.04877

どちらの群のサイズが変化しても傾向は同じ（検出力は落ちる）

サンプルサイズに依存せず、Type I Error の確率は変わらなさそう。

# Links
- http://www.u.tsukuba.ac.jp/~hirai.akiyo.ft/meeting13.files/SLAA_5_16%28withoutimage%29.pdf